In [1]:
# Load libraries
import datetime as dt
from datetime import datetime
import time 
import calendar
start_time = datetime.now()

import pandas as pd
import numpy as np

import scipy.stats as sps
import random

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from xgboost import XGBClassifier


print('LOAD DURATION:',datetime.now() - start_time)

LOAD DURATION: 0:00:00.793367


In [2]:
%%time
# load the data
df = pd.read_csv('train_test.csv')
oot = pd.read_csv('oot.csv')

CPU times: user 1.38 s, sys: 297 ms, total: 1.67 s
Wall time: 1.68 s


In [3]:
X = df.drop(columns=['fraud_label'])
y = df[['fraud_label']]

oot_x = oot.drop(columns=['fraud_label'])
oot_y = oot['fraud_label']

In [4]:
# create functions to calculate fdr
def fdr_cal(x_data, y_data, model_choice):
    model = model_choice
    pop = int(round(len(x_data)*0.03))
    result = pd.DataFrame(model.predict_proba(x_data),columns=['prob_0', 'prob_1'])
    temp = x_data.copy()
    temp['fraud_label'] = y_data
    temp['prob_1']= list(result.prob_1)
    temp0 = temp.sort_values('prob_1', ascending=False)
    temp1 = temp0.head(pop)
    fdr = temp1.fraud_label.sum() / y_data.sum()
    
    return fdr

In [5]:
# create a table to store the data
xgb_table = pd.DataFrame()
num_tree = [1000,1500,1800]
max_depth = 5
learn_rate = [0.02,0.05]



i=0
for lr in learn_rate:
        for num in num_tree:
                xgb_table.loc[i,'n_estimators'] = num
                xgb_table.loc[i,'learning_rate'] = lr
                xgb_table.loc[i,'max_depth'] = max_depth
                i+=1



xgb_table['Train'] = 0.000
xgb_table['Test'] = 0.000
xgb_table['OOT'] = 0.000

In [6]:
# create a for loop to calculate all Xgboost FDR@3%

# using KFold
n_choice = 10
kf = KFold(n_splits=n_choice)


num_tree = [1000,1500,1800]
max_depth = 5
learn_rate = [0.02,0.05]

i=0
for lr in learn_rate:
        for num in num_tree:
            for train_index, test_index in kf.split(X):
                X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
                y_train, y_test = y.iloc[train_index, :].values.ravel(), y.iloc[test_index, :].values.ravel()

                xgb = XGBClassifier(n_estimators=num, max_depth=max_depth, use_label_encoder=False,
                         learning_rate=lr, objective='binary:logistic', verbosity = 0, n_jobs = -1)

                xgb.fit(X_train,y_train)

                xgb_table.loc[(xgb_table['n_estimators']==num)\
                              &(xgb_table['learning_rate']==lr)\
                              ,'Train'] += fdr_cal(X_train, y_train, xgb)
                xgb_table.loc[(xgb_table['n_estimators']==num)\
                              &(xgb_table['learning_rate']==lr)\
                              ,'Test'] += fdr_cal(X_test, y_test, xgb)
                xgb_table.loc[(xgb_table['n_estimators']==num)\
                              &(xgb_table['learning_rate']==lr)\
                              ,'OOT'] += fdr_cal(oot_x, oot_y, xgb)
                i+=1
        print(f'Iteration {i} is completed.')

Iteration 30 is completed.
Iteration 60 is completed.


In [7]:
for col in ['Train','Test','OOT']:
    xgb_table[col] = xgb_table[col]/n_choice

In [8]:
# save results
xgb_table.to_excel('xgb_table_results.xlsx')

In [9]:
xgb_table

,n_estimators,learning_rate,max_depth,Train,Test,OOT
0,1000.0,0.02,5.0,0.580761,0.573975,0.555407
1,1500.0,0.02,5.0,0.582569,0.573564,0.555658
2,1800.0,0.02,5.0,0.583876,0.573646,0.555784
3,1000.0,0.05,5.0,0.586011,0.573299,0.555490
4,1500.0,0.05,5.0,0.589407,0.572744,0.554652
5,1800.0,0.05,5.0,0.590643,0.571720,0.554359
